In [127]:
from datasets import load_dataset
import json
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import re

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\norbe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
dataset = load_dataset('poloclub/diffusiondb', '2m_first_5k')

Generating train split: 0 examples [00:00, ? examples/s]

Dataset diffusiondb downloaded and prepared to C:/Users/norbe/.cache/huggingface/datasets/poloclub___diffusiondb/2m_first_5k/0.9.1/547894e3a57aa647ead68c9faf148324098f47f2bc1ab6705d670721de9d89d1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [128]:
num_rows = dataset['train'].num_rows

In [129]:
stop_words = set(stopwords.words('english'))
min_word_length = 2
min_word_freq = 10

In [130]:
custom_word_map = {}
for i in range(num_rows):
    prompt = dataset['train']['prompt'][i]
    clean_prompt = re.sub('[^A-Za-z]+',' ', prompt)
    clean_prompt_arr = clean_prompt.split(' ')
    for word in clean_prompt_arr:
        if len(word) >= min_word_length and word not in stop_words:
            if word in custom_word_map.keys():
                custom_word_map[word] += 1
            else: 
                custom_word_map[word] = 1

In [131]:
with open('WORDMAP_coco_5_cap_per_img_5_min_word_freq.json') as json_file:
    existing_word_map = json.load(json_file)

In [132]:
clean_existing_word_map = {}
for key, value in existing_word_map.items():
    clean_word = re.sub('[^A-Za-z]+',' ', key)
    if len(clean_word) >= min_word_length and clean_word not in stop_words:
        clean_existing_word_map[clean_word] = value

In [133]:
special_tokens = {
    "<pad>": 0,
    "<unk>": 1,
    "<start>": 2,
    "<end>": 3,
}

In [134]:
final_word_map = special_tokens | clean_existing_word_map | custom_word_map

In [135]:
for i, (key, value) in enumerate(final_word_map.items()):
    final_word_map[key] = i

In [136]:
print("Final length of word map:", len(final_word_map))

Final length of word map: 13806


In [137]:
with open('word_map.json', 'w') as fp:
    json.dump(final_word_map, fp)